# Gradient Descent

Optimization algorithms are the cornerstone of machine learning, because many learning tasks can be recast as an optimization problem.

One of the most widely spread optimization routines is gradient descent (also known as steepest descent), which is an iterative optimization method. Gradient descent seeks to *find a local minimum of a function* using only the information from the gradient (i.e., it performs first-order optimization). If we are *maximizing* a function instead of minimizing, we use gradient ascent.

The idea behing gradient descent is simple. It relies on the fact that *the gradient of a function points in the direction of the greatest rate of increase of that function*. Thus, if we follow the negative gradient, we will walk towards the direction of the greatest rate of decrease. By following the negative gradient, we will end up converging to a (local) minimum of the function. In contrast, by following the gradient, we will converge to a (local) maximum of the function.

Consider that we want to minimize a function $f(x)$ with respect to the variable $x$. The variable $x$ can be a vector. The gradient of the function, $\nabla_x f(x)$, is a vector of the same length of $x$ that points to the direction of steepest ascent. Assume that we have an initial guess $x_0$ of where the minimum might be (in practice, the initial guess $x_0$ can be chosen depending on the available information, or even randomly). We can refine $x_0$ by following the negative gradient at that point, $\nabla_x f(x_0)$:
$$ x_1 = x_0-\rho_0 \nabla_x f(x_0) $$
Here, $\rho_0$ is a scalar that denotes the stepsize. For now, let us simply consider that $\rho_0$ is "small enough." Then, $x_1$ can be closer to the actual local minimum than the initial guess. We can apply this process iteratively to further refine the solution as
$$ x_{t+1} = x_t-\rho_t \nabla_x f(x_t) $$

## Example 1

As a simple initial example, consider that $x=[x_0, x_1]^\top$ is a 2-dimensional vector, and the function $f(x)$ is given by
$$ f(x) = \left(x_1-\frac{5.1}{4\pi^2}x_0^2+\frac{5}{\pi}x_0-6\right)^2+10\left(1-\frac{1}{8\pi}\right)\cos(x_0)+10.$$
This complicated function is called "Branin function."

**Visualize how this function looks like.** The code below:

- Evaluates $f(x)$.
- Defines auxiliary python functions for plotting.
- Evaluates $f(x)$ on a grid of points and plot it.

In [ ]:
# Import useful packages
from math import pi
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

# Define the function f(x) for 2-dimensional vector x
def f(x):
    t = 1.0/(8*pi)
    s = 10.0
    r = 6.0
    c = 5.0/pi
    b = 5.1/(4*pi**2)
    a = 1.0
    term1 = a*(x[1] - b*x[0]**2 + c*x[0] - r)**2
    term2 = s*(1.0-t)*np.cos(x[0])
    return(term1 + term2 + s)

# Define auxiliary functions for plotting
def plot_surf(X, Y, Z):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, 
                           cmap=plt.cm.RdBu, linewidth=0, antialiased=False)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.xlabel('$x_0$')
    plt.ylabel('$x_1$')
    plt.show()
    
def plot_contour(X, Y, Z, xpath=None, ypath=None):
    fig = plt.figure()
    cs = plt.contourf(X, Y, Z, 100, cmap=plt.cm.RdBu)
    fig.colorbar(cs, shrink=0.5, aspect=5)
    plt.xlabel('$x_0$')
    plt.ylabel('$x_1$')
    if xpath is not None:
        plt.plot(xpath, ypath)
    plt.plot([-pi,pi,9.42478],[12.275,2.275,2.475],'kx')
    plt.show()

In [ ]:
# Create a grid on the region -5<x0<12, 0<x1<15
Xlin = np.linspace(-5,12,100)
Ylin = np.linspace(0,15,100)
# Grid of points (x,y)
Xgrid, Ygrid = np.meshgrid(Xlin, Ylin)

# Evaluate the function f on all the point of the grid     
Zgrid = np.array([[f((x,y)) for x in Xlin] for y in Ylin])

# Surface plot
plot_surf(Xgrid, Ygrid, Zgrid)

# Contour plot
plot_contour(Xgrid, Ygrid, Zgrid)

**Local minima.** This function has three minima: $x=(-\pi,12.275)$, $x=(\pi,2.275)$ and $x=(9.42478,2.475)$. It turns out that the three local minima are also global minima, since the three of them reach the same function value of $f(x)=0.397887$.

**Gradient (mathematical expression).** But for now, let us assume that we are not aware of this information, and we want to minimize the function using gradient ascent. Indeed, real-world examples have higher dimensionality and we will not be able to make these plots. For that, we first obtain the gradient of the function, which is given by
$$
\nabla_x f(x) =
\left[ 
\begin{array}{c}
\frac{\partial f(x)}{\partial x_0} \\
\frac{\partial f(x)}{\partial x_1} \\
\end{array}
\right]
=
\left[ 
\begin{array}{c}
2\left(x_1-\frac{5.1}{4\pi^2}x_0^2+\frac{5}{\pi}x_0-6\right)\left( -\frac{5.1}{2\pi^2}x_0+\frac{5}{\pi} \right)-10\left(1-\frac{1}{8\pi}\right)\sin(x_0) \\
2\left(x_1-\frac{5.1}{4\pi^2}x_0^2+\frac{5}{\pi}x_0-6\right) \\
\end{array}
\right]
$$

**Gradient (python).** Here is the function that evaluates the gradient of $f(x)$ at any point $x$:

In [ ]:
# Gradient evaluation
def grad_f(x):
    t = 1.0/(8*pi)
    s = 10.0
    r = 6.0
    c = 5.0/pi
    b = 5.1/(4*pi**2)
    a = 1.0
    term1 = 2.0*a*(x[1] - b*x[0]**2 + c*x[0] - r)
    term2 = -s*(1.0-t)*np.sin(x[0])
    
    y = np.zeros((2,))
    y[0] = term1*(-2.0*b*x[0] + c) + term2
    y[1] = term1
    return y

**Initial guess.** We start with an initial guess of, say, $x_0 = (0,0)$.

**Algorithm.** At each iteration $t$ of the gradient descent algorithm, we set
$$x_{t+1} = x_t-\rho_t \nabla_x f(x_t).$$
For simplicity, we assume a constant step size $\rho_t=\rho$ for all iterations.

**Convergence.** How many iterations should we run? There is not a unique answer for that. One standard approach is to stop when the gradient has norm of approximately zero (this indicates that we are near the optimum). Another approach is to stop when $x_{t+1}$ and $x_t$ are approximately equal (this indicates that we are barely moving away from that point). We will follow the former approach. It is also common to stop the algorithm after reaching a pre-specified number of iterations to prevent infinite loops.

**Implementation of gradient descent in Python.**

In [ ]:
# Set the initial guess
x = np.zeros(2)

# Set the step size (constant in this case)
rho = 0.1

# Define the tolerance (to declare convergence)
tol = 1.0e-6

# Specify the maximum number of iterations
max_iterations = 1000

# Iteration counter
it = 0

# While not converged, repeat
while(it==0 or np.sqrt(np.sum(grad**2))>tol and it<max_iterations):
    # Evaluate the gradient
    grad = grad_f(x)
    # Refine the initial guess
    x = x - rho*grad
    # Increase the iteration number
    it += 1

**[Task]** Run the code above. In the cell below, print the value of the optimum $x$ found, as well as the value of $f(x)$ evaluated at that point. Does this point correspond to any of the global optima described above?

**Plotting the optimization path.** Now, keep track of *optimization path* to visualize what gradient ascent is actually doing.

In [ ]:
# Lists to save the optimization path
x0_path = []
x1_path = []

# Set the step size (constant in this case)
rho = 0.1

# Initial value
x = (0.0,0.0)

# Append the initial value to the path
x0_path.append(x[0])
x1_path.append(x[1])

# Specify the maximum number of iterations
max_iterations = 1000

# Iteration counter
it = 0

# While not converged, repeat
while(it==0 or np.sqrt(np.sum(grad**2))>tol and it<max_iterations):
    # Evaluate the gradient
    grad = grad_f(x)
    # Refine the initial guess
    x = x - rho*grad
    # Append the value to the path
    x0_path.append(x[0])
    x1_path.append(x[1])
    # Increase the iteration number
    it += 1

plot_contour(Xgrid, Ygrid, Zgrid, x0_path, x1_path)

**[Tasks]** Based on the code above, perform the following tasks and answer these questions:

1. Play with *smaller* step sizes and visualize the optimization path in each case. Does the path look smoother or sharper? Why?

2. Play with *slightly larger* values of the step size ($\rho=0.2$, $0.3$, and $0.4$). What happens?

3. Set $\rho=0.1$ again. Initialize the algorithm at $x=(6.0,14.0)$. Plot the optimization path.

4. Think of a setting (initial guess and step size) such that the solution is $x=(-\pi,12.275)$. How can you get this solution? Verify that by plotting the optimization path.

5. Set the initial guess at $x=(-2.0,10.0)$ and the step size as $\rho=0.089$. Run the code again. What can you see?

## Example 2

Consider the following function of three variables,
$$
f(x) =  \frac{1}{2}\left( x_0^4+ x_1^4 + x_2^4 \right) -8\left( x_0^2+x_1^2+x_2^2 \right) +\frac{5}{2}\left(x_0 +x_1+ x_2\right),
$$
with gradient
$$
\nabla_x f(x) =
\left[ 
\begin{array}{c}
\frac{\partial f(x)}{\partial x_0} \\
\frac{\partial f(x)}{\partial x_1} \\
\frac{\partial f(x)}{\partial x_2}
\end{array}
\right]
=
\left[ 
\begin{array}{c}
2x_0^3-16x_0+\frac{5}{2} \\
2x_1^3-16x_1+\frac{5}{2} \\
2x_2^3-16x_2+\frac{5}{2}
\end{array}
\right].
$$

(This is called the Styblinski-Tang function.)

**[Task]** Implement a gradient descent method to find a local minimum of this function. Use a step size of $\rho=0.01$. How many different local optima can you find?

*Note: You do not need to plot the optimization path here. In general, we do not plot the optimization path when the dimensionality of $x$ is larger than 2.* 